In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import requests
import urllib.request
import time

authors = []
dates = []
statements = []
content = []
sources = []
targets = []
  

In [ ]:
def scrape_website(page_number):
  pgno = str(page_number)
  URL = 'https://www.politifact.com/factchecks/list/?page='+ pgno + '&category=coronavirus'
  webpage = requests.get(URL)
  soup = BeautifulSoup(webpage.text,'html.parser')
  #Get location
  statement_footer = soup.find_all('footer',attrs ={'class':'m-statement__footer'})
  statement_quote = soup.find_all('div',attrs={'class':'m-statement__quote'})
  statement_meta = soup.find_all('div',attrs={'class':'m-statement__meta'})
  target = soup.find_all('div',attrs={'class':'m-statement__meter'})

  for i in statement_footer:
    link1 = i.text.strip()
    name_and_date = link1.split()
    first_name = name_and_date[1]
    last_name = name_and_date[2]
    full_name = first_name+' '+last_name
    month = name_and_date[4]
    day = name_and_date[5]
    year = name_and_date[6]
    date = month + ' '+ day + ' '+year
    dates.append(date)
    authors.append(full_name)

  for i in statement_quote:
    link2 = i.find_all('a')
    statement_text = link2[0].text.strip()
    statements.append(statement_text)
    atag=i.find_all('a')[0]
    link2 = i.find_all('a')
    statement_text = link2[0].text.strip()
    URL2 = 'https://www.politifact.com' +  atag['href']
    webpage = requests.get(URL2)
    soup2 = BeautifulSoup(webpage.text)
    statement_content = soup2.find_all('article',attrs={'class':'m-textblock'})
    content.append(''.join(statement_content[0].text.strip().splitlines()))


  for i in statement_quote:
   atag=i.find_all('a')[0]   

  for i in statement_meta:
    link3 = i.find_all('a')
    source_text = link3[0].text.strip()
    sources.append(source_text)

  for i in target:
      link4 = i.find('div',attrs={'class':'c-image'}).find('img').get('alt')
      targets.append(link4)





In [ ]:
n = 50
for i in range(1,n):
  scrape_website(i)

In [ ]:
data = pd.DataFrame(columns = ['author','statement','article','source','date','target'])
data['author'] = pd.Series(authors)
data['statement']= pd.Series(statements)
data['source']= pd.Series(sources)
data['article']= pd.Series(content)
data['date']= pd.Series(dates)
data['target']= pd.Series(targets)

data

,author,statement,article,source,date,target
0,Ciara O'Rourke,“100% of COVID deaths in Canada now due to mRN...,Canada’s government has not published data sho...,Viral image,"February 10, 2023",false
1,Amy Sherman,Joe Biden closed schools during the COVID-19 p...,President Joe Biden said during his State of t...,Lauren Boebert,"February 8, 2023",false
2,Tom Kertscher,"""Unusual cancers” are “exploding right into st...",A video shared on Facebook tied COVID-19 vacci...,Facebook posts,"February 8, 2023",false
3,Madison Czopek,“Pfizer just got caught doing gain of function...,"Over three years of the pandemic, scientists h...",Instagram posts,"February 6, 2023",false
4,Paul Specht,Diamond and Silk pundit died from COVID-19,"When conservative media pundit Lynette ""Diamon...",Tweets,"February 6, 2023",false
...,...,...,...,...,...,...
1440,Daniel Funke,"Says a ""Clorox bottle claimed it could kill (t...",Some social media users are publishing photos ...,Facebook posts,"January 30, 2020",false
1441,Daniel Funke,"Says the Chinese Communist Party ""will finally...",A website with ties to Steve Bannon and a Chin...,Bloggers,"January 28, 2020",false
1442,Tom Kertscher,"""CORONAVIRUS: Reports of 10,000 DEAD in Wuhan,...",A claim circulating on Facebook suggests a cor...,Facebook posts,"January 28, 2020",pants-fire
1443,Daniel Funke,"Says China ""stole Coronavirus from Canada and ...",A widely shared article on social media inaccu...,Bloggers,"January 28, 2020",false


In [ ]:
data.to_csv('New_CoronaFake.csv')